#### Praktikum 4

Klasifikasi dengan ANN

In [15]:
from google.colab import drive # menggunakan Google Drive sebagai penyimpanan berkas dan alat komputasi
drive.mount('/content/drive') # menghubungkan Google Colab dengan Google Drive Anda

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import numpy as np # menyediakan dukungan untuk array multidimensi dan berbagai fungsi matematika
import pandas as pd # menyediakan struktur data dan fungsi operasi data
import tensorflow as tf # merupakan salah satu pustaka yang sangat populer untuk deep learning dan machine learning

In [17]:
dataset = pd.read_csv('/content/drive/MyDrive/Machine_Learning_2023/week_09/Churn_Modelling.csv') # Mengambil dataset
X = dataset.iloc[:, 3:-1].values # Mengambil nilai untuk variabel X
y = dataset.iloc[:, -1].values # Mengambil label atau nilai untuk variabel y

print(X) # Menampilkan nilai X

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [18]:
from sklearn.preprocessing import LabelEncoder # mengubah data kategori menjadi data numerik
le = LabelEncoder() # membuat objek LabelEncoder
X[:, 2] = le.fit_transform(X[:, 2]) # melakukan perubahan atribut dengan nilai numerik

In [19]:
from sklearn.compose import ColumnTransformer # memungkinkan untuk mengonfigurasi transformasi berbeda pada kolom berbeda dalam DataFrame
from sklearn.preprocessing import OneHotEncoder # mengambil variabel kategori dan mengonversinya menjadi vektor biner
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough') # menggabungkan beberapa transformasi dalam satu langkah
X = np.array(ct.fit_transform(X)) # melakukan perubahan nilai (encoding) pada X dan memformatnya ke array numpy

In [20]:
from sklearn.model_selection import train_test_split # Digunakan untuk memisahkan data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0) # Membagi data uji dan latih dengan rasio data uji 80% dan data test 20%

In [21]:
from sklearn.preprocessing import StandardScaler # Membantu dalam standarisasi fitur
sc = StandardScaler() # Membuat objek StandardScaler
X_train = sc.fit_transform(X_train) # melakukan standarisasi pada data uji X_train
X_test = sc.transform(X_test) # melakukan standarisasi pada data tes X_test

#### Membuat Model ANN

In [22]:
ann = tf.keras.models.Sequential() # Membuat model neural network Sequential

ann.add(tf.keras.layers.Dense(units=6, activation='relu')) # menambahkan sebuah layer fully connected, atau dikenal juga sebagai dense layer
# Fungsi aktivasi Rectified Linear Unit (ReLU) menghasilkan input secara langsung jika positif; jika tidak, menghasilkan nol.

ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) # menambahkan sebuah layer fully connected (dense layer)
# Fungsi aktivasi Sigmoid mengubah setiap nilai output menjadi nilai antara 0 dan 1, yang bisa diinterpretasikan sebagai probabilitas.

#### Training Model

In [23]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # Optimizer ADAM (Adaptive Moment Estimation) untuk memperbarui bobot
# binary_crossentropy, sebagai loss function untuk klasifikasi binari
# ‘accuracy’ sebagai metrik yang menghitung persentase prediksi

In [24]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100) # Melatih model ANN

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5332 - accuracy: 0.7961
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4681 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 0.4481 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4369 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4284 - accuracy: 0.7964
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4184 - accuracy: 0.8076
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4056 - accuracy: 0.8274
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3903 - accuracy: 0.8369
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3759 - accuracy: 0.8445
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3653 - accura

Membuat Prediksi

Diberikan informasi sebagai berikut,

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: $50000

Apakah customer tersebut perlu dipertahankan?

In [25]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) # Menampilkan hasil prediksi

1/1 [==============================] - 0s 126ms/step
[[False]]


In [26]:
y_pred = ann.predict(X_test) # melakukan prediksi nilai berdasarkan X_test
y_pred = (y_pred > 0.5) # digunakan dalam konteks klasifikasi biner ketika Anda ingin mengonversi nilai-nilai prediksi menjadi label kelas
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)) # menggabungkan dan menampilkan 2 array

63/63 [==============================] - 0s 5ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#### Cek Akurasi dan Confusion Matrix

In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score # Berguna dalam menghitung akurasi skor dan confusion matrix
cm = confusion_matrix(y_test, y_pred) # menghitung confusion matrix berdasarkan hasil tes dan prediksi
print(cm) # menampilkan hasil confusion matrix
accuracy_score(y_test, y_pred) # menghitung dan menampilkan akurasi matrix berdasarkan hasil tes dan prediksi

[[1521   74]
 [ 199  206]]


0.8635